In [1]:
import os 
import openai
from dotenv import load_dotenv 
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [2]:
load_dotenv()

True

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [4]:
gemma_model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
gemma_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020A88785CC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020A888C5AB0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
gpt_model = ChatGroq(model="openai/gpt-oss-120b", groq_api_key=groq_api_key)
gpt_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020A86AACCA0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020A888F07C0>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello, How are you?")
]
messages

[SystemMessage(content='Translate the following from English to French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, How are you?', additional_kwargs={}, response_metadata={})]

In [7]:
gemma_result = gemma_model.invoke(messages)
gemma_result

AIMessage(content="Bonjour, comment allez-vous ? \n\n\nLet me know if you have any other phrases you'd like translated!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 22, 'total_tokens': 49, 'completion_time': 0.049090909, 'prompt_time': 0.001323699, 'queue_time': 0.187896396, 'total_time': 0.050414608}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--063d7c4e-26e0-48a8-8d52-5f172a7b7795-0', usage_metadata={'input_tokens': 22, 'output_tokens': 27, 'total_tokens': 49})

In [8]:
gpt_result = gpt_model.invoke(messages)
gpt_result

AIMessage(content='Bonjour, comment ça va\u202f?', additional_kwargs={'reasoning_content': 'We need to translate "Hello, How are you?" to French. The translation: "Bonjour, comment ça va ?" or "Bonjour, comment allez-vous ?" Probably the standard. The user just asked for translation. So respond with the translation.'}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 87, 'total_tokens': 153, 'completion_time': 0.120550522, 'prompt_time': 0.004749003, 'queue_time': 0.084982749, 'total_time': 0.125299525}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_085e8f07e2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a2f5428c-9626-44f7-9a80-7cc5369fa36f-0', usage_metadata={'input_tokens': 87, 'output_tokens': 66, 'total_tokens': 153})

In [9]:
parser = StrOutputParser()

In [10]:
gemma_output = parser.invoke(gemma_result)
gemma_output

"Bonjour, comment allez-vous ? \n\n\nLet me know if you have any other phrases you'd like translated!\n"

In [11]:
gpt_output = parser.invoke(gpt_result)
gpt_output

'Bonjour, comment ça va\u202f?'

In [12]:
gemma_chain = gemma_model|parser
gemma_chain_output = gemma_chain.invoke(messages)
gemma_chain_output

'Here are a few ways to translate "Hello, How are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal)\n* **Bonjour, ça va ?** (Informal) \n\n\nThe best option depends on the context and your relationship with the person you\'re speaking to. \n'

In [13]:
gpt_chain = gpt_model|parser
gpt_chain_output = gpt_chain.invoke(messages)
gpt_chain_output

'Bonjour, comment ça va\u202f?'

In [14]:
generic_template = "Translate the following into {language} : "
prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)
prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language} : '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [15]:
prompt_result = prompt.invoke({"language":"French", "text":"Hello"})
prompt_result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French : ', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [16]:
prompt_result.to_messages()

[SystemMessage(content='Translate the following into French : ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [17]:
final_gemma_chain = prompt|gemma_model|parser
final_gemma_chain_output = final_gemma_chain.invoke({"language":"French", "text":"Hello"})
final_gemma_chain_output

'Bonjour \n'

In [18]:
final_gpt_chain = prompt|gpt_model|parser
final_gpt_chain_output = final_gpt_chain.invoke({"language":"French", "text":"Hello"})
final_gpt_chain_output

'Bonjour'